In [123]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN

In [139]:
df = pd.read_csv("data/temp_merged.csv")

Index(['DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes',
       'OriginAirportID', 'DestAirportID', 'CRSArrTime', 'ArrDel15',
       'ArrDelayMinutes', 'windspeedKmph_x', 'DewPointF_x', 'cloudcover_x',
       'precipMM_x', 'pressure_x', 'WindGustKmph_x', 'visibility_x',
       'weatherCode_x', 'tempF_x', 'WindChillF_x', 'winddirDegree_x',
       'humidity_x', 'windspeedKmph_y', 'DewPointF_y', 'cloudcover_y',
       'precipMM_y', 'pressure_y', 'WindGustKmph_y', 'visibility_y',
       'weatherCode_y', 'tempF_y', 'WindChillF_y', 'winddirDegree_y',
       'humidity_y'],
      dtype='object')

In [125]:
df = df.drop(columns=['Unnamed: 0', 'FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth',
       'Origin', 'Dest','ArrTime'])
y = df['ArrDel15']
X = df.drop(columns=['ArrDel15','ArrDelayMinutes'])

In [126]:
#FEATURE SET
X.columns

Index(['DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes',
       'OriginAirportID', 'DestAirportID', 'CRSArrTime', 'windspeedKmph_x',
       'DewPointF_x', 'cloudcover_x', 'precipMM_x', 'pressure_x',
       'WindGustKmph_x', 'visibility_x', 'weatherCode_x', 'tempF_x',
       'WindChillF_x', 'winddirDegree_x', 'humidity_x', 'windspeedKmph_y',
       'DewPointF_y', 'cloudcover_y', 'precipMM_y', 'pressure_y',
       'WindGustKmph_y', 'visibility_y', 'weatherCode_y', 'tempF_y',
       'WindChillF_y', 'winddirDegree_y', 'humidity_y'],
      dtype='object')

In [127]:
#SMOTE OVERSAMPLING
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
sm = ADASYN(sampling_strategy='minority')
x_train, y_train = sm.fit_sample(x_train, y_train)
sampled_df = pd.concat([pd.DataFrame(x_train), pd.DataFrame(y_train)], axis=1)

k=X.columns
z=pd.DataFrame(y,columns=['ArrDel15'])
sampled_df.columns=k.append(z.columns)

In [128]:
sampled_df.ArrDel15.value_counts()

1.0    1035405
0.0    1024063
Name: ArrDel15, dtype: int64

In [112]:
#Using selectKBest to select best features
"""
min_max_scaler = preprocessing.MinMaxScaler()
Scaled_X = min_max_scaler.fit_transform(x_train)

bestfeatures = SelectKBest(chi2, k=15)
fit = bestfeatures.fit(Scaled_X,y_train)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
feats = list(featureScores.nlargest(22,'Score').Specs)
x_train = sampled_df[feats]
x_test = x_test[feats]

"""

In [129]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# LOGISTIC REGRESSION

In [130]:
lr = linear_model.LogisticRegression(max_iter=2000)
clf = lr.fit(x_train, y_train)

In [131]:
print("Logistic regression Train Accuracy : ", clf.score(x_train,y_train))
print("Logistic regression Test Accuracy  : ", metrics.accuracy_score(y_test, lr.predict(x_test)))

print(confusion_matrix(y_test,lr.predict(x_test),labels=[1,0]))

print(classification_report(y_test,lr.predict(x_test)))

Logistic regression Train Accuracy :  0.9162795147850737
Logistic regression Test Accuracy  :  0.9157337558288173
[[ 79753  36752]
 [ 10052 428873]]
              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    438925
         1.0       0.89      0.68      0.77    116505

    accuracy                           0.92    555430
   macro avg       0.90      0.83      0.86    555430
weighted avg       0.91      0.92      0.91    555430



# RANDOM FOREST CLASSIFIER

In [136]:
rf = RandomForestClassifier(criterion='entropy')
clf = rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

In [137]:
print(" Train Accuracy : ", clf.score(x_train, y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test, y_pred, labels=[1,0]))

print(classification_report(y_test, y_pred))

 Train Accuracy :  0.9999714506833703
 Test Accuracy  :  0.9183389446014799
CONFUSION MATRIX
 [[ 81689  34816]
 [ 10541 428384]]
              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    438925
         1.0       0.89      0.70      0.78    116505

    accuracy                           0.92    555430
   macro avg       0.91      0.84      0.87    555430
weighted avg       0.92      0.92      0.91    555430



# EXTRA TREES CLASSIFIER

In [134]:
ex = ExtraTreesClassifier(criterion='entropy')
clf = ex.fit(x_train,y_train)
y_pred = ex.predict(x_test)

In [135]:
print(" Train Accuracy : ", clf.score(x_train,y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test,y_pred,labels=[1,0]))

print(classification_report(y_test,y_pred))

 Train Accuracy :  0.9999992283968479
 Test Accuracy  :  0.9112831499918982
CONFUSION MATRIX
 [[ 86501  30004]
 [ 19272 419653]]
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94    438925
         1.0       0.82      0.74      0.78    116505

    accuracy                           0.91    555430
   macro avg       0.88      0.85      0.86    555430
weighted avg       0.91      0.91      0.91    555430



#  XGBOOST CLASSIFIER

In [132]:
xg = XGBClassifier()
clf = xg.fit(x_train, y_train)
y_pred = xg.predict(x_test)

In [133]:
print(" Train Accuracy : ", clf.score(x_train, y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test, y_pred, labels=[1,0]))

print(classification_report(y_test, y_pred))

 Train Accuracy :  0.922017155824485
 Test Accuracy  :  0.9192787569990818
CONFUSION MATRIX
 [[ 81080  35425]
 [  9410 429515]]
              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    438925
         1.0       0.90      0.70      0.78    116505

    accuracy                           0.92    555430
   macro avg       0.91      0.84      0.87    555430
weighted avg       0.92      0.92      0.92    555430

